# Determining Sentence Sentiment using NLP

In [1]:
import torch
import pandas as pd

# My libraries
from src.data_loader import SSTDataPipeline
from src.trainer import Trainer
from src.models import DynamicRNN
from src.plotter import Plotter

# Check Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running experiments on hardware type: {DEVICE}")

Running experiments on hardware type: cpu


In [2]:
BATCH_SIZE = 64
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 5  # SST-5
DROPOUT = 0.5
EPOCHS = 15
LEARNING_RATE = 0.001

# Pre-trained Vector Path
VECTOR_PATH = "./data/vector.txt"

In [3]:
# Initialize Data Pipeline (with vectors)
print("Initializing Data Pipeline...")
data_pipe = SSTDataPipeline(
    vector_path=VECTOR_PATH, batch_size=BATCH_SIZE, device=DEVICE
)
train_iter, val_iter, test_iter = data_pipe.run()

# Get Vocabulary Info
VOCAB_SIZE = data_pipe.vocab_size
PAD_IDX = data_pipe.get_pad_idx()
PRETRAINED_VECTORS = data_pipe.get_embeddings()

print(f"Data ready. Vocab Size: {VOCAB_SIZE}, Pad Index: {PAD_IDX}")

Initializing Data Pipeline...
--> Loading SST splits...
downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:01<00:00, 514kB/s] 


extracting
--> Building Vocabulary...
    Loading specific vectors: ./data/vector.txt
    Success: Vocab built with './data/vector.txt'
    Label Mapping: {'very negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very positive': 4}
    Vocab Size: 16581
--> Creating Iterators...
Data ready. Vocab Size: 16581, Pad Index: 1


### Part 1: Find the best model

In [5]:
experiments_config = [
    # --- Naive RNN ---
    {
        "name": "1_RNN_Random",
        "rnn_type": "rnn",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": False,
    },
    {
        "name": "2_RNN_Pretrained",
        "rnn_type": "rnn",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": True,
    },
    # --- Naive LSTM ---
    {
        "name": "3_LSTM_Random",
        "rnn_type": "lstm",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": False,
    },
    {
        "name": "4_LSTM_Pretrained",
        "rnn_type": "lstm",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": True,
    },
    # --- Naive GRU ---
    {
        "name": "5_GRU_Random",
        "rnn_type": "gru",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": False,
    },
    {
        "name": "6_GRU_Pretrained",
        "rnn_type": "gru",
        "n_layers": 1,
        "bidirectional": False,
        "use_pretrained": True,
    },
    # --- Better LSTM (Bi-Directional, 2 Layers) ---
    {
        "name": "7_BiLSTM_Deep_Random",
        "rnn_type": "lstm",
        "n_layers": 2,
        "bidirectional": True,
        "use_pretrained": False,
    },
    {
        "name": "8_BiLSTM_Deep_Pretrained",
        "rnn_type": "lstm",
        "n_layers": 2,
        "bidirectional": True,
        "use_pretrained": True,
    },
    # --- Better GRU (Bi-Directional, 2 Layers) ---
    {
        "name": "9_BiGRU_Deep_Random",
        "rnn_type": "gru",
        "n_layers": 2,
        "bidirectional": True,
        "use_pretrained": False,
    },
    {
        "name": "10_BiGRU_Deep_Pretrained",
        "rnn_type": "gru",
        "n_layers": 2,
        "bidirectional": True,
        "use_pretrained": True,
    },
]

In [ ]:
# Store all results here
all_histories = {}  # Key: Model Name, Value: History Dict
all_summaries = []  # List of Summary Dicts for DataFrame

for config in experiments_config:
    print(f"\n{'='*20} Running: {config['name']} {'='*20}")

    # 1. Init Model
    model = DynamicRNN(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        output_dim=OUTPUT_DIM,
        n_layers=config["n_layers"],
        bidirectional=config["bidirectional"],
        dropout=DROPOUT,
        pad_idx=PAD_IDX,
        rnn_type=config["rnn_type"],
    )

    # 2. Embeddings
    if config["use_pretrained"]:
        if PRETRAINED_VECTORS is not None:
            print(f"Loading vectors...")
            model.embedding.weight.data.copy_(PRETRAINED_VECTORS)
        else:
            print("Vectors not found, using random.")

    # 3. Hyperparams for record keeping
    hyperparams = {
        "Type": config["rnn_type"].upper(),
        "BiDir": config["bidirectional"],
        "Embeds": "Pretrained" if config["use_pretrained"] else "Random",
        "Layers": config["n_layers"],
    }

    # 4. Train
    trainer = Trainer(model, DEVICE)
    history, summary = trainer.run_experiment(
        train_iter,
        val_iter,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        name=config["name"],
        hyperparameters=hyperparams,
        save_weights=True,
    )

    # 5. Store Data
    all_histories[config["name"]] = history
    all_summaries.append(summary)

print("\nAll models trained successfully.")


==================== Running: 1_RNN_Random ====================
Starting 1_RNN_Random | Params: 5,118,433
  Epoch: 01 | Time: 0m 7s | Train Accuracy: 0.256 | Val Accuracy: 0.261
  Epoch: 02 | Time: 0m 7s | Train Accuracy: 0.283 | Val Accuracy: 0.298
  Epoch: 03 | Time: 0m 7s | Train Accuracy: 0.316 | Val Accuracy: 0.318
  Epoch: 04 | Time: 0m 8s | Train Accuracy: 0.345 | Val Accuracy: 0.328
  Epoch: 05 | Time: 0m 7s | Train Accuracy: 0.378 | Val Accuracy: 0.333
  Epoch: 06 | Time: 0m 7s | Train Accuracy: 0.403 | Val Accuracy: 0.332
  Epoch: 07 | Time: 0m 7s | Train Accuracy: 0.432 | Val Accuracy: 0.316
  Epoch: 08 | Time: 0m 8s | Train Accuracy: 0.460 | Val Accuracy: 0.320
  Epoch: 09 | Time: 0m 7s | Train Accuracy: 0.483 | Val Accuracy: 0.337
  Epoch: 10 | Time: 0m 8s | Train Accuracy: 0.497 | Val Accuracy: 0.313
  Epoch: 11 | Time: 0m 8s | Train Accuracy: 0.524 | Val Accuracy: 0.323
  Epoch: 12 | Time: 0m 8s | Train Accuracy: 0.547 | Val Accuracy: 0.318
  Epoch: 13 | Time: 0m 8s | T

In [ ]:
print(f"Generating plots for {len(all_histories)} models...\n")

# You can adjust grid size or plotting logic here
for model_name, history in all_histories.items():
    Plotter.plot_history(history, title=model_name)

In [ ]:
df_results = pd.DataFrame(all_summaries)

# Reorder columns for readability (optional)
cols = [
    "Model",
    "Type",
    "BiDir",
    "Embeds",
    "Best Val Acc",
    "Best Val Loss",
    "Time (s)",
    "Parameters",
]
# Filter to ensure columns exist before selecting
cols = [c for c in cols if c in df_results.columns]
df_results = df_results[cols + [c for c in df_results.columns if c not in cols]]

# Display
display(df_results.sort_values(by="Best Val Acc", ascending=False))

# Export
csv_path = "sst_experiment_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")